In [1]:
import requests
import pandas as pd
import sqlite3
import logging
import time
from typing import List, Dict

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('wb_parser.log'),
        logging.StreamHandler()
    ]
)

def init_db():
    conn = sqlite3.connect('wb_products.db')
    cursor = conn.cursor()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        query TEXT NOT NULL,
        page INTEGER NOT NULL,
        article TEXT UNIQUE NOT NULL,
        name TEXT,
        brand TEXT,
        price REAL NOT NULL DEFAULT 0,
        sale_price REAL NOT NULL DEFAULT 0,
        discount REAL DEFAULT 0,
        rating REAL DEFAULT 0,
        feedbacks INTEGER DEFAULT 0,
        promo TEXT DEFAULT 'Нет акции'
    )
    ''')
    
    conn.commit()
    return conn

def save_to_db(conn, products: List[Dict]):
    cursor = conn.cursor()
    saved = 0
    
    for product in products:
        try:
            # Проверяем, что артикул есть (обязательное поле)
            if not product.get('Артикул'):
                logging.warning(f"Пропуск товара без артикула: {product}")
                continue

            # Подготавливаем данные, заменяя None на значения по умолчанию
            data = (
                product.get('Запрос', ''),
                product.get('Страница', 1),
                str(product.get('Артикул', '')),
                product.get('Название', ''),
                product.get('Бренд', ''),
                float(product.get('Цена (руб)', 0)),
                float(product.get('Цена со скидкой (руб)', 0)),
                float(product.get('Скидка (%)', 0)),
                float(product.get('Рейтинг', 0)),
                int(product.get('Отзывы', 0)),
                product.get('Акция', 'Нет акции')
            )
            
            cursor.execute('''
            INSERT OR REPLACE INTO products 
            (query, page, article, name, brand, price, sale_price, discount, rating, feedbacks, promo)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', data)
            
            saved += 1
        except Exception as e:
            logging.error(f"Ошибка сохранения товара {product.get('Артикул', '')}: {str(e)}")
            logging.error(f"Данные товара: {product}")
    
    conn.commit()
    return saved

def parse_wb_search(query: str, pages: int = 1) -> pd.DataFrame:
    """Парсинг данных с Wildberries"""
    conn = init_db()
    all_products = []
    
    for page in range(1, pages + 1):
        try:
            url = "https://search.wb.ru/exactmatch/ru/common/v4/search"
            params = {
                "query": query, "page": page, "dest": -1257786,
                "regions": "80,64,38,4,115,83,33,68,70,69,30,86,75,40,1,66,48,110,31,22,71,114",
                "resultset": "catalog", "sort": "popular", "spp": 30, "limit": 100
            }
            
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
            
            response = requests.get(url, headers=headers, params=params, timeout=15)
            response.raise_for_status()
            data = response.json()
            
            products = []
            for product in data.get('data', {}).get('products', []):
                price = product.get('priceU', 0) / 100
                sale_price = product.get('salePriceU', price * 100) / 100
                
                if price <= 0:
                    continue
                    
                product_info = {
                    'Название': product.get('name', ''),
                    'Бренд': product.get('brand', ''),
                    'Цена (руб)': price,
                    'Цена со скидкой (руб)': sale_price,
                    'Скидка (%)': round((1 - sale_price / price) * 100, 2) if price else 0,
                    'Рейтинг': product.get('reviewRating', 0),
                    'Отзывы': product.get('feedbacks', 0),
                    'Артикул': str(product.get('id', '')),
                    'Акция': product.get('promoText', 'Нет акции'),
                    'Страница': page,
                    'Запрос': query
                }
                products.append(product_info)
            
            if products:
                saved = save_to_db(conn, products)
                all_products.extend(products)
                logging.info(f"Страница {page}: сохранено {saved}/{len(products)} товаров")
            
            time.sleep(1.5)
            
        except Exception as e:
            logging.error(f"Ошибка страницы {page}: {str(e)}")
    
    conn.close()
    
    if all_products:
        df = pd.DataFrame(all_products)
        columns_order = ['Запрос', 'Страница', 'Артикул', 'Название', 'Бренд', 
                        'Цена (руб)', 'Цена со скидкой (руб)', 'Скидка (%)',
                        'Рейтинг', 'Отзывы', 'Акция']
        return df[columns_order]
    
    return pd.DataFrame()

def main():
    print("=== Парсер Wildberries ===")
    query = input("Введите поисковый запрос: ").strip()
    pages = min(int(input("Сколько страниц парсить (1-50): ") or 1), 50)
    
    start_time = time.time()
    df = parse_wb_search(query, pages)
    
    if not df.empty:
        excel_file = f"wb_{query}_products.xlsx"
        df.to_excel(excel_file, index=False)
        
        print(f"\nРезультаты:")
        print(f"1. Сохранено в базу данных: wb_products.db")
        print(f"2. Экспортировано в Excel: {excel_file}")
        print(f"\nВсего товаров: {len(df)}")
        print(f"Пример данных:\n{df.head(3).to_string(index=False)}")
    else:
        print("\nДанные не получены. Проверьте wb_parser.log")
    
    print(f"\nВремя выполнения: {time.time() - start_time:.2f} сек")

if __name__ == "__main__":
    main()

=== Парсер Wildberries ===


Введите поисковый запрос:  ноутбук
Сколько страниц парсить (1-50):  3


2025-06-29 15:59:53,379 - INFO - Страница 1: сохранено 100/100 товаров
2025-06-29 15:59:55,035 - INFO - Страница 2: сохранено 100/100 товаров
2025-06-29 15:59:56,700 - INFO - Страница 3: сохранено 100/100 товаров



Результаты:
1. Сохранено в базу данных: wb_products.db
2. Экспортировано в Excel: wb_ноутбук_products.xlsx

Всего товаров: 300
Пример данных:
 Запрос  Страница   Артикул                                                     Название Бренд  Цена (руб)  Цена со скидкой (руб)  Скидка (%)  Рейтинг  Отзывы     Акция
ноутбук         1 271469851 Ноутбук для работы и учебы 15 6 4-ядра INTEL N5095 16+256 ГБ           27000.0                18304.0       32.21      4.6    1104 Нет акции
ноутбук         1 250901053      Ноутбук для работы 14.1" AMD Ryzen 5 3500U 8Gb SSD256Gb  Acer     25979.0                18330.0       29.44      4.6      16 Нет акции
ноутбук         1 251828731   Ноутбук для учебы и работы HTEX H16 Pro 4-Ядра 16ГБ 512 ГБ           49500.0                16947.0       65.76      4.8    1121 Нет акции

Время выполнения: 5.12 сек


In [2]:
import sqlite3
conn = sqlite3.connect('wb_products.db')
pd.read_sql("SELECT query, name, price, sale_price, discount, rating, feedbacks FROM products LIMIT 5", conn)

,query,name,price,sale_price,discount,rating,feedbacks
0,ноутбук,Ноутбуки M543 AMD Ryzen 3 4300U 8ГБ+512ГБ FHD ...,53791.0,28287.0,47.41,4.8,39
1,ноутбук,i5-1240P ноутбук игровой 128 2ТБ для работы 16...,59551.0,29859.0,49.86,5.0,1
2,ноутбук,"15.6"" Vivobook 15 X1504ZA Intel Core i5-1235U ...",58567.0,43846.0,25.14,5.0,7
3,ноутбук,"Ноутбук AZ-1406 14"", Intel N3350, 6Gb, 64Gb eM...",18240.0,12658.0,30.60,0.0,0
4,ноутбук,"Игровой ноутбук для работы и учёбы, 16+1024, W...",61498.0,18589.0,69.77,0.0,0


In [3]:
import sqlite3

def check_db_structure():
    conn = sqlite3.connect('wb_products.db')
    cursor = conn.cursor()
    
    # Проверяем существование таблицы
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='products'")
    table_exists = cursor.fetchone()
    
    if not table_exists:
        print("Таблица 'products' не существует!")
        return
    
    # Получаем структуру таблицы
    cursor.execute("PRAGMA table_info(products)")
    columns = cursor.fetchall()
    
    print("\nСтруктура таблицы 'products':")
    print("ID|Name|Type|NotNull|Default|PK")
    for col in columns:
        print(col)
    
    # Проверяем наличие обязательных полей
    required_columns = {'article', 'name', 'brand', 'price', 'rating'}
    existing_columns = {col[1] for col in columns}
    
    missing = required_columns - existing_columns
    if missing:
        print(f"\nОтсутствуют обязательные колонки: {missing}")
    else:
        print("\nВсе обязательные колонки присутствуют")
    
    conn.close()

check_db_structure()


Структура таблицы 'products':
ID|Name|Type|NotNull|Default|PK
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'query', 'TEXT', 1, None, 0)
(2, 'page', 'INTEGER', 1, None, 0)
(3, 'article', 'TEXT', 1, None, 0)
(4, 'name', 'TEXT', 0, None, 0)
(5, 'brand', 'TEXT', 0, None, 0)
(6, 'price', 'REAL', 1, '0', 0)
(7, 'sale_price', 'REAL', 1, '0', 0)
(8, 'discount', 'REAL', 0, '0', 0)
(9, 'rating', 'REAL', 0, '0', 0)
(10, 'feedbacks', 'INTEGER', 0, '0', 0)
(11, 'promo', 'TEXT', 0, "'Нет акции'", 0)

Все обязательные колонки присутствуют


In [4]:
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_batch
from typing import List, Dict
import pandas as pd
import requests
import logging
import time

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('wb_parser.log'),
        logging.StreamHandler()
    ]
)

# Конфигурация PostgreSQL (замените значения на свои)
POSTGRES_CONFIG = {
    'dbname': 'wb_products',      # Название вашей БД
    'user': 'postgres',           # Ваш пользователь PostgreSQL
    'password': '123456',  # Пароль пользователя
    'host': 'localhost',          # Хост, где работает PostgreSQL
    'port': '5432'                # Порт PostgreSQL
}

def init_db():
    """Инициализация подключения к PostgreSQL"""
    try:
        conn = psycopg2.connect(**POSTGRES_CONFIG)
        cursor = conn.cursor()
        
        # Создание таблицы, если она не существует
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS products (
            id SERIAL PRIMARY KEY,
            query TEXT NOT NULL,
            page INTEGER NOT NULL,
            article TEXT NOT NULL,
            name TEXT,
            brand TEXT,
            price NUMERIC(10, 2) NOT NULL DEFAULT 0,
            sale_price NUMERIC(10, 2) NOT NULL DEFAULT 0,
            discount NUMERIC(5, 2) DEFAULT 0,
            rating NUMERIC(3, 2) DEFAULT 0,
            feedbacks INTEGER DEFAULT 0,
            promo TEXT DEFAULT 'Нет акции',
            CONSTRAINT unique_article UNIQUE (article)
        )
        ''')
        
        conn.commit()
        return conn
    except Exception as e:
        logging.error(f"Ошибка подключения к PostgreSQL: {str(e)}")
        raise

def save_to_db(conn, products: List[Dict]):
    """Сохранение товаров в PostgreSQL"""
    cursor = conn.cursor()
    saved = 0
    
    # Подготовка данных для вставки
    data_to_insert = []
    for product in products:
        if not product.get('Артикул'):
            logging.warning(f"Пропуск товара без артикула: {product}")
            continue
            
        data_to_insert.append((
            product.get('Запрос', ''),
            product.get('Страница', 1),
            str(product.get('Артикул', '')),
            product.get('Название', ''),
            product.get('Бренд', ''),
            float(product.get('Цена (руб)', 0)),
            float(product.get('Цена со скидкой (руб)', 0)),
            float(product.get('Скидка (%)', 0)),
            float(product.get('Рейтинг', 0)),
            int(product.get('Отзывы', 0)),
            product.get('Акция', 'Нет акции')
        ))
    
    if not data_to_insert:
        return 0
    
    try:
        # Используем execute_batch для эффективной пакетной вставки
        execute_batch(cursor, '''
        INSERT INTO products 
        (query, page, article, name, brand, price, sale_price, discount, rating, feedbacks, promo)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (article) DO UPDATE SET
            query = EXCLUDED.query,
            page = EXCLUDED.page,
            name = EXCLUDED.name,
            brand = EXCLUDED.brand,
            price = EXCLUDED.price,
            sale_price = EXCLUDED.sale_price,
            discount = EXCLUDED.discount,
            rating = EXCLUDED.rating,
            feedbacks = EXCLUDED.feedbacks,
            promo = EXCLUDED.promo
        ''', data_to_insert)
        
        saved = len(data_to_insert)
        conn.commit()
    except Exception as e:
        conn.rollback()
        logging.error(f"Ошибка сохранения товаров в БД: {str(e)}")
        logging.error(f"Первые 5 товаров с ошибкой: {data_to_insert[:5]}")
    
    return saved

# Остальные функции (parse_wb_search, main) остаются без изменений
def parse_wb_search(query: str, pages: int = 1) -> pd.DataFrame:
    """Парсинг данных с Wildberries"""
    conn = init_db()
    all_products = []
    
    for page in range(1, pages + 1):
        try:
            url = "https://search.wb.ru/exactmatch/ru/common/v4/search"
            params = {
                "query": query, "page": page, "dest": -1257786,
                "regions": "80,64,38,4,115,83,33,68,70,69,30,86,75,40,1,66,48,110,31,22,71,114",
                "resultset": "catalog", "sort": "popular", "spp": 30, "limit": 100
            }
            
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
            
            response = requests.get(url, headers=headers, params=params, timeout=15)
            response.raise_for_status()
            data = response.json()
            
            products = []
            for product in data.get('data', {}).get('products', []):
                price = product.get('priceU', 0) / 100
                sale_price = product.get('salePriceU', price * 100) / 100
                
                if price <= 0:
                    continue
                    
                product_info = {
                    'Название': product.get('name', ''),
                    'Бренд': product.get('brand', ''),
                    'Цена (руб)': price,
                    'Цена со скидкой (руб)': sale_price,
                    'Скидка (%)': round((1 - sale_price / price) * 100, 2) if price else 0,
                    'Рейтинг': product.get('reviewRating', 0),
                    'Отзывы': product.get('feedbacks', 0),
                    'Артикул': str(product.get('id', '')),
                    'Акция': product.get('promoText', 'Нет акции'),
                    'Страница': page,
                    'Запрос': query
                }
                products.append(product_info)
            
            if products:
                saved = save_to_db(conn, products)
                all_products.extend(products)
                logging.info(f"Страница {page}: сохранено {saved}/{len(products)} товаров")
            
            time.sleep(1.5)
            
        except Exception as e:
            logging.error(f"Ошибка страницы {page}: {str(e)}")
    
    conn.close()
    
    if all_products:
        df = pd.DataFrame(all_products)
        columns_order = ['Запрос', 'Страница', 'Артикул', 'Название', 'Бренд', 
                        'Цена (руб)', 'Цена со скидкой (руб)', 'Скидка (%)',
                        'Рейтинг', 'Отзывы', 'Акция']
        return df[columns_order]
    
    return pd.DataFrame()

def main():
    print("=== Парсер Wildberries ===")
    query = input("Введите поисковый запрос: ").strip()
    pages = min(int(input("Сколько страниц парсить (1-50): ") or 1), 50)
    
    start_time = time.time()
    df = parse_wb_search(query, pages)
    
    if not df.empty:
        excel_file = f"wb_{query}_products.xlsx"
        df.to_excel(excel_file, index=False)
        
        print(f"\nРезультаты:")
        print(f"1. Сохранено в базу данных PostgreSQL: {POSTGRES_CONFIG['dbname']}")
        print(f"2. Экспортировано в Excel: {excel_file}")
        print(f"\nВсего товаров: {len(df)}")
        print(f"Пример данных:\n{df.head(3).to_string(index=False)}")
    else:
        print("\nДанные не получены. Проверьте wb_parser.log")
    
    print(f"\nВремя выполнения: {time.time() - start_time:.2f} сек")

if __name__ == "__main__":
    main()

=== Парсер Wildberries ===


Введите поисковый запрос:  ноутбук
Сколько страниц парсить (1-50):  3


2025-06-29 16:00:12,381 - INFO - Страница 1: сохранено 100/100 товаров
2025-06-29 16:00:14,071 - INFO - Страница 2: сохранено 100/100 товаров
2025-06-29 16:00:15,739 - INFO - Страница 3: сохранено 100/100 товаров



Результаты:
1. Сохранено в базу данных PostgreSQL: wb_products
2. Экспортировано в Excel: wb_ноутбук_products.xlsx

Всего товаров: 300
Пример данных:
 Запрос  Страница   Артикул                                                     Название Бренд  Цена (руб)  Цена со скидкой (руб)  Скидка (%)  Рейтинг  Отзывы     Акция
ноутбук         1 271469851 Ноутбук для работы и учебы 15 6 4-ядра INTEL N5095 16+256 ГБ           27000.0                18304.0       32.21      4.6    1104 Нет акции
ноутбук         1 250901053      Ноутбук для работы 14.1" AMD Ryzen 5 3500U 8Gb SSD256Gb  Acer     25979.0                18330.0       29.44      4.6      16 Нет акции
ноутбук         1 251828731   Ноутбук для учебы и работы HTEX H16 Pro 4-Ядра 16ГБ 512 ГБ           49500.0                16947.0       65.76      4.8    1121 Нет акции

Время выполнения: 5.10 сек


In [5]:
import pandas as pd
from sqlalchemy import create_engine
from tabulate import tabulate
import matplotlib.pyplot as plt

# Конфигурация PostgreSQL
POSTGRES_CONFIG = {
    'dbname': 'wb_products',
    'user': 'postgres',
    'password': '123456',
    'host': 'localhost',
    'port': '5432'
}

def fetch_data_from_db():
    """Получение данных из PostgreSQL"""
    try:
        connection_string = f"postgresql://{POSTGRES_CONFIG['user']}:{POSTGRES_CONFIG['password']}@{POSTGRES_CONFIG['host']}:{POSTGRES_CONFIG['port']}/{POSTGRES_CONFIG['dbname']}"
        engine = create_engine(connection_string)
        
        query = """
        SELECT query, name, price, sale_price, discount, rating, feedbacks 
        FROM products 
        WHERE query = 'ноутбук'
        ORDER BY price DESC
        LIMIT 5
        """
        
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Ошибка при подключении к PostgreSQL: {e}")
        return pd.DataFrame()

def display_beautiful_table(df):
    """Вывод красивой таблицы с настройками"""
    if not df.empty:
        # Подготовка данных
        df = df.rename(columns={
            'query': 'Запрос',
            'name': 'Название',
            'price': 'Цена',
            'sale_price': 'Цена со скидкой',
            'discount': 'Скидка (%)',
            'rating': 'Рейтинг',
            'feedbacks': 'Отзывы'
        })
        
        # Форматирование чисел
        df['Цена'] = df['Цена'].apply(lambda x: f"{x:,.1f} ₽")
        df['Цена со скидкой'] = df['Цена со скидкой'].apply(lambda x: f"{x:,.1f} ₽")
        df['Скидка (%)'] = df['Скидка (%)'].apply(lambda x: f"{x:.2f}%")
        df['Рейтинг'] = df['Рейтинг'].apply(lambda x: f"{x:.1f} ★")
        
        # Настройки отображения
        plt.rcParams['font.size'] = 12
        pd.set_option('display.max_colwidth', 40)
        
        # Создание красивой таблицы
        table = tabulate(
            df,
            headers='keys',
            tablefmt='grid',
            stralign='center',
            numalign='center',
            showindex=False,
            colalign=('center', 'left', 'right', 'right', 'right', 'right', 'right')
        )
        
        # Вывод таблицы
        print("\n" + "="*120)
        print("СПИСОК НОУТБУКОВ".center(120))
        print("="*120)
        print(table)
        print("="*120 + "\n")
        
    else:
        print("Данные не найдены.")

if __name__ == "__main__":
    print("\nЗагрузка данных...")
    data = fetch_data_from_db()
    display_beautiful_table(data)


Загрузка данных...

                                                    СПИСОК НОУТБУКОВ                                                    
+----------+------------------------------------------------------------+-------------+-------------------+--------------+-----------+----------+
|  Запрос  | Название                                                   |        Цена |   Цена со скидкой |   Скидка (%) |   Рейтинг |   Отзывы |
+==========+============================================================+=============+===================+==============+===========+==========+
| ноутбук  | i7-1255U Ноутбук игровой 16.1" 16ГБ+512ГБ NVIDIA GeForce   | 207,400.0 ₽ |        43,119.0 ₽ |       79.21% |     5.0 ★ |       22 |
+----------+------------------------------------------------------------+-------------+-------------------+--------------+-----------+----------+
| ноутбук  | Ноутбук для работы, учебы и игр 15,6" IPS 16 GB SSD 512 GB | 145,000.0 ₽ |        37,488.0 ₽ |       74.15% |     4

In [6]:
from fastapi import FastAPI, HTTPException, Query
from typing import Optional
import psycopg2
from psycopg2.extras import RealDictCursor
import uvicorn

app = FastAPI()

# Конфигурация PostgreSQL
POSTGRES_CONFIG = {
    'dbname': 'wb_products',
    'user': 'postgres',
    'password': '123456',
    'host': 'localhost',
    'port': '5432'
}

def get_db_connection():
    """Создание подключения к PostgreSQL"""
    try:
        conn = psycopg2.connect(
            dbname=POSTGRES_CONFIG['dbname'],
            user=POSTGRES_CONFIG['user'],
            password=POSTGRES_CONFIG['password'],
            host=POSTGRES_CONFIG['host'],
            port=POSTGRES_CONFIG['port'],
            cursor_factory=RealDictCursor
        )
        return conn
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Database connection error: {str(e)}")

@app.get("/api/products/")
def get_products(
    min_price: Optional[float] = Query(None, description="Минимальная цена товара"),
    max_price: Optional[float] = Query(None, description="Максимальная цена товара"),
    min_rating: Optional[float] = Query(None, description="Минимальный рейтинг товара (0-5)"),
    max_rating: Optional[float] = Query(None, description="Максимальный рейтинг товара (0-5)"),
    min_feedbacks: Optional[int] = Query(None, description="Минимальное количество отзывов"),
    max_feedbacks: Optional[int] = Query(None, description="Максимальное количество отзывов"),
    limit: int = Query(10, description="Количество возвращаемых товаров", ge=1, le=100),
    offset: int = Query(0, description="Смещение для пагинации", ge=0)
):
    """
    Получение списка товаров с возможностью фильтрации по цене, рейтингу и количеству отзывов.
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        query = """
            SELECT id, query, page, article, name, brand, 
                   price, sale_price, discount, rating, feedbacks, promo
            FROM products
            WHERE 1=1
        """
        params = []
        
        if min_price is not None:
            query += " AND price >= %s"
            params.append(min_price)
        if max_price is not None:
            query += " AND price <= %s"
            params.append(max_price)
        if min_rating is not None:
            query += " AND rating >= %s"
            params.append(min_rating)
        if max_rating is not None:
            query += " AND rating <= %s"
            params.append(max_rating)
        if min_feedbacks is not None:
            query += " AND feedbacks >= %s"
            params.append(min_feedbacks)
        if max_feedbacks is not None:
            query += " AND feedbacks <= %s"
            params.append(max_feedbacks)
        
        query += " ORDER BY price DESC LIMIT %s OFFSET %s"
        params.extend([limit, offset])
        
        cursor.execute(query, params)
        products = cursor.fetchall()
        
        count_query = "SELECT COUNT(*) FROM products WHERE 1=1"
        count_params = params[:-2]
        
        if count_params:
            cursor.execute(count_query + query.split("WHERE 1=1")[1].split("ORDER BY")[0], count_params)
        else:
            cursor.execute(count_query)
            
        total = cursor.fetchone()['count']
        
        conn.close()
        
        return {
            "success": True,
            "data": products,
            "meta": {
                "total": total,
                "limit": limit,
                "offset": offset,
                "filters": {
                    "min_price": min_price,
                    "max_price": max_price,
                    "min_rating": min_rating,
                    "max_rating": max_rating,
                    "min_feedbacks": min_feedbacks,
                    "max_feedbacks": max_feedbacks
                }
            }
        }
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Для запуска в Jupyter Notebook
if __name__ == "__main__":
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    await server.serve()  # Только в асинхронном окружении

INFO:     Started server process [14528]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14528]


In [ ]:
import requests
import pandas as pd

# 1. Отправка запроса
response = requests.get(
    url="http://localhost:8000/api/products/",
    params={
        "min_price": 50000,
        "min_rating": 4.5,
        "limit": 100
    }
)

# 2. Проверка ответа
if response.status_code == 200:
    data = response.json()
    
    # 3. Конвертация в DataFrame
    df = pd.DataFrame(data['data'])
    
    # 4. Сохранение в Excel
    df.to_excel("filtered_products.xlsx", index=False)
    print(f"Сохранено {len(df)} товаров")
    
    # 5. Анализ данных
    print(f"\nСредняя цена: {df['price'].mean():.2f} руб.")
    print(f"Лучший товар: {df.loc[df['rating'].idxmax()]['name']}")
    
else:
    print(f"Ошибка: {response.status_code}", response.text)